# 灰色关联分析法 Grey Relational Analysis

FinCreWorld & xyfJASON

## 1 简述

> 参考资料：[数学建模笔记——评价类模型之灰色关联分析 - 小白的文章 - 知乎](https://zhuanlan.zhihu.com/p/161536409)

灰色关联分析的研究对象是一个系统，系统的发展受多个因素的影响，我们想知道这些因素中哪些影响大、哪些影响小。

如果把系统和因素都量化为数值，那么我们研究的就是多个序列的关联程度，或者换句话说，这些序列的**曲线几何形状的相似程度**。形状越相似，说明关联度越高。我们只需要量化各个因素的曲线 $\{x_i\}$ 与系统的曲线 $\{x_0\}$ 之间的相似程度，就能刻画出各个因素对系统的影响程度。

也就是说，我们应用灰色关联分析法的时候，需要一个母序列（参考数列） $\{x_0\}$ 和若干子序列（比较数列） $\{x_i\}$。

可是这和我们的主题——综合评价与决策有什么关系呢？在做评价类问题时，我们需要人为构造这个母序列——**将各个指标中最好的数据合并**作为「理想解」，然后计算各评价对象关于这个理想解的灰色关联度，关联度越高，评价对象越好。

## 2 步骤

1. 确定比较对象（评价对象）和参考数列（评价标准）

   设有 $m$ 个评价对象，$n$ 个评价指标，则参考数列为 $x_0=\{x_0(k)|k=1,2,\cdots,n\}$，比较数列为

   $$
   x_i=\{x_i(k)|k=1,2,\cdots,n\},i=1,2,\cdots,m
   $$
   
   注意所有的指标都需要**正向化**。


2. 确定各指标值对应的权重
   
   $$
   w=[w_1,\cdots,w_n]
   $$


3. 计算灰色关联系数
   
   $$
   \xi_i(k)=\frac{\min\limits_s\min\limits_t|x_0(t)-x_s(t)|+\rho\max\limits_s\max\limits_t|x_0(t)-x_s(t)|}
   {|x_0(k)-x_i(k)|+\rho\max\limits_s\max\limits_t|x_0(t)-x_s(t)|}
   $$
   
   称 'minmin' 项为两级最小差，'maxmax' 项为两级最大差，$\rho$ 为分辨系数，$\rho$ 越大，分辨率越大


4. 计算灰色加权关联度。其计算公式为
   
   $$
   r_i=\sum\limits_{k=1}^nw_i\xi_i(k)
   $$
   
   $r_i$ 为第 $i$ 个评价对象对理想对象的灰色加权关联度。


5. 评价分析

   根据灰色加权关联度的大小，对各评价对象进行排序，可以建立评价对象的关联序，关联度越大，评价结果越好。

## 3 代码模板

见同文件夹下的 `evaluation.py` 模块。

## 4 实例

在 6 个待选的零部件供应商中选择一个合作伙伴，各待选供应商有关数据如下

In [1]:
import pandas as pd

df = pd.DataFrame([[ 0.83 , 0.9  , 0.99 , 0.92 , 0.87 , 0.95 ],
     [ 326  , 295  , 340  , 287  , 310  , 303  ],
     [ 21   , 38   , 25   , 19   , 27   , 10   ],
     [ 3.2  , 2.4  , 2.2  , 2    , 0.9  , 1.7  ],
     [ 0.2  , 0.25 , 0.12 , 0.33 , 0.2  , 0.09 ],
     [ 0.15 , 0.2  , 0.14 , 0.09 , 0.15 , 0.17 ],
     [ 250  , 180  , 300  , 200  , 150  , 175  ],
     [ 0.23 , 0.15 , 0.27 , 0.3  , 0.18 , 0.26 ],
     [ 0.87 , 0.95 , 0.99 , 0.89 , 0.82 , 0.94 ]])
df.columns = [['待选供应商']*6, list('123456')]
df.index = ['产品质量', '产品价格', '地理位置', '售后服务', '技术水平', '经济效益', '供应能力', '市场影响度', '交货情况']
df

待选供应商                                        
            1       2       3       4       5       6
产品质量     0.83    0.90    0.99    0.92    0.87    0.95
产品价格   326.00  295.00  340.00  287.00  310.00  303.00
地理位置    21.00   38.00   25.00   19.00   27.00   10.00
售后服务     3.20    2.40    2.20    2.00    0.90    1.70
技术水平     0.20    0.25    0.12    0.33    0.20    0.09
经济效益     0.15    0.20    0.14    0.09    0.15    0.17
供应能力   250.00  180.00  300.00  200.00  150.00  175.00
市场影响度    0.23    0.15    0.27    0.30    0.18    0.26
交货情况     0.87    0.95    0.99    0.89    0.82    0.94

其中产品质量、技术水平、供应能力、经济效益、交货情况、市场影响度指标为效益型指标，其标准化方式为

$$
std = \frac{ori - \min(ori)}{\max(ori) - \min(ori)}
$$

而产品地位、地理位置、售后服务指标属于成本型指标，其标准化方式为

$$
std = \frac{\max(ori) - ori}{\max(ori) - \min(ori)}
$$

In [2]:
# 对数据进行预处理
import numpy as np
from evaluation import positive_scale

res = positive_scale(x=df.T.values, kind=np.array([0,1,1,1,0,0,0,0,0]))
df.iloc[:, :] = res.T
df

待选供应商                                                  
              1         2         3         4         5         6
产品质量   0.000000  0.437500  1.000000  0.562500  0.250000  0.750000
产品价格   0.264151  0.849057  0.000000  1.000000  0.566038  0.698113
地理位置   0.607143  0.000000  0.464286  0.678571  0.392857  1.000000
售后服务   0.000000  0.347826  0.434783  0.521739  1.000000  0.652174
技术水平   0.458333  0.666667  0.125000  1.000000  0.458333  0.000000
经济效益   0.545455  1.000000  0.454545  0.000000  0.545455  0.727273
供应能力   0.666667  0.200000  1.000000  0.333333  0.000000  0.166667
市场影响度  0.533333  0.000000  0.800000  1.000000  0.200000  0.733333
交货情况   0.294118  0.764706  1.000000  0.411765  0.000000  0.705882

In [3]:
# 假设各项指标地位相同，权重相等
# 计算关联度
from evaluation import GreyRelationalAnalysis

solver = GreyRelationalAnalysis(x=df.values.T)
R, r = solver.run()
r

array([0.46299243, 0.55599428, 0.64907363, 0.6527227 , 0.493562  ,
       0.61301504])

In [4]:
# 列表
df2 = pd.DataFrame(np.concatenate((R.T, r.reshape(1, -1)), axis=0))
df2.columns = [f'供应商{i}' for i in '123456']
df2.index = [*[f'指标{i}' for i in range(1, 10)], 'r']
df2

,供应商1,供应商2,供应商3,供应商4,供应商5,供应商6
指标1,0.333333,0.470588,1.000000,0.533333,0.400000,0.666667
指标2,0.404580,0.768116,0.333333,1.000000,0.535354,0.623529
指标3,0.560000,0.333333,0.482759,0.608696,0.451613,1.000000
指标4,0.333333,0.433962,0.469388,0.511111,1.000000,0.589744
指标5,0.480000,0.600000,0.363636,1.000000,0.480000,0.333333
指标6,0.523810,1.000000,0.478261,0.333333,0.523810,0.647059
指标7,0.600000,0.384615,1.000000,0.428571,0.333333,0.375000
指标8,0.517241,0.333333,0.714286,1.000000,0.384615,0.652174
指标9,0.414634,0.680000,1.000000,0.459459,0.333333,0.629630
r,0.462992,0.555994,0.649074,0.652723,0.493562,0.613015
